In [31]:
#%%
print('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting')

from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#%%
seed = 42
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train.csv', delimiter = ',')
train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train_gene.csv', delimiter = ',')
train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_train_hml.csv')
train_target = train_target[['EMB_MIC']]
# don't touch test data, split out validation data from training data during training
# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_EMB/aa_data_test_pca4k.csv', delimiter = ',')
# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test.csv', delimiter = ',')
test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test_gene.csv', delimiter = ',')
test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_test_hml.csv')
test_target = test_target[['EMB_MIC']]

all_data = np.concatenate((train_data, test_data), axis=0)
all_target = pd.concat((train_target, test_target), axis=0)

# train_data, val_data, train_target, val_target = train_test_split(all_data, all_target, test_size=0.2, random_state=42, stratify=all_target)

/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting


In [37]:
y_test.shape

(2060, 1)

# RF

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, mean_absolute_error, accuracy_score

# Example data (replace with your actual data)
# Assuming X and y are your feature and target dataframes respectively
np.random.seed(42)

# Convert y to a series for compatibility with sklearn
y = train_target
X = train_data
# Define the model
model = RandomForestClassifier(random_state=42)

# Set up K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store results for each fold
confusion_matrices = []
maes = []
accuracies = []

# Cross-validation loop
for train_index, test_index in kf.split(X):
    X_train, X_test = train_data[train_index], train_data[test_index]
    y_train, y_test = train_target.iloc[train_index], train_target.iloc[test_index]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate confusion matrix, MAE, and accuracy
    cm = confusion_matrix(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results
    confusion_matrices.append(cm)
    maes.append(mae)
    accuracies.append(accuracy)
    
    # Print results for the current fold
    print(f"Fold results:\nConfusion Matrix:\n{cm}\nMAE: {mae:.4f}\nAccuracy: {accuracy:.4f}\n")

# Print average results over all folds
print(f"Average MAE: {np.mean(maes):.4f}")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")

/tmp/ipykernel_57797/64894018.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Fold results:
Confusion Matrix:
[[1369   48   44]
 [  75  103   85]
 [  31   84  221]]
MAE: 0.2146
Accuracy: 0.8218



/tmp/ipykernel_57797/64894018.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Fold results:
Confusion Matrix:
[[1407   41   35]
 [  79  114   85]
 [  28   67  204]]
MAE: 0.1932
Accuracy: 0.8374



/tmp/ipykernel_57797/64894018.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Fold results:
Confusion Matrix:
[[1419   57   31]
 [  68  111   69]
 [  24   67  214]]
MAE: 0.1801
Accuracy: 0.8466



/tmp/ipykernel_57797/64894018.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Fold results:
Confusion Matrix:
[[1375   43   18]
 [  80  117   79]
 [  37   84  227]]
MAE: 0.1922
Accuracy: 0.8345



/tmp/ipykernel_57797/64894018.py:31: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Fold results:
Confusion Matrix:
[[1360   50   30]
 [  83  114   94]
 [  30   85  214]]
MAE: 0.2097
Accuracy: 0.8194

Average MAE: 0.1980
Average Accuracy: 0.8319


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, mean_absolute_error, accuracy_score

# Example data (replace with your actual data)
# Assuming X and y are your feature and target dataframes respectively
np.random.seed(42)

# Convert y to a series for compatibility with sklearn
y = train_target
X = train_data

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy']
}

# Define the model
model = RandomForestClassifier(random_state=42)

# Set up K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to find the best parameters
grid_search.fit(X, y)

# Print the best parameters and best score
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

# Evaluate the best model using cross-validation
best_model = grid_search.best_estimator_

# Lists to store results for each fold
confusion_matrices = []
maes = []
accuracies = []

# Cross-validation loop
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train the best model
    best_model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = best_model.predict(X_test)
    
    # Calculate confusion matrix, MAE, and accuracy
    cm = confusion_matrix(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results
    confusion_matrices.append(cm)
    maes.append(mae)
    accuracies.append(accuracy)
    
    # Print results for the current fold
    print(f"Fold results:\nConfusion Matrix:\n{cm}\nMAE: {mae:.4f}\nAccuracy: {accuracy:.4f}\n")

# Print average results over all folds
print(f"Average MAE: {np.mean(maes):.4f}")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")


In [10]:
for _ in kf.split(X):
    print(_)
    break


(array([   1,    2,    3, ..., 9192, 9194, 9196]), array([   0,    8,   12, ..., 9190, 9193, 9195]))


# MLP

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, mean_absolute_error, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, mean_absolute_error, accuracy_score

# Load the data
# train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train.csv', delimiter=',')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_train_hml3.csv')
# train_target = train_target[['EMB_MIC']].values.ravel()

# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test.csv', delimiter=',')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_test_hml3.csv')
# test_target = test_target[['EMB_MIC']].values.ravel()

# Combine train and test data for splitting
all_data = np.concatenate((train_data, test_data), axis=0)
all_target = np.concatenate((train_target, test_target), axis=0)

# Split the data into training and validation sets
train_data, val_data, train_target, val_target = train_test_split(all_data, all_target, test_size=0.2, random_state=42, stratify=all_target)

# Initialize the MLP model
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)

# Train the model
mlp.fit(train_data, train_target)

# Evaluate on the validation set
val_predictions = mlp.predict(val_data)
val_confusion_matrix = confusion_matrix(val_target, val_predictions)
val_mae = mean_absolute_error(val_target, val_predictions)
val_accuracy = accuracy_score(val_target, val_predictions)

print("Validation Confusion Matrix:\n", val_confusion_matrix)
print("Validation MAE: {:.4f}".format(val_mae))
print("Validation Accuracy: {:.4f}".format(val_accuracy))

# Evaluate on the test set
test_predictions = mlp.predict(test_data)
test_confusion_matrix = confusion_matrix(test_target, test_predictions)
test_mae = mean_absolute_error(test_target, test_predictions)
test_accuracy = accuracy_score(test_target, test_predictions)

print("Test Confusion Matrix:\n", test_confusion_matrix)
print("Test MAE: {:.4f}".format(test_mae))
print("Test Accuracy: {:.4f}".format(test_accuracy))


/mnt/storageG1/lwang/miniconda3/envs/ml-workshop/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Validation Confusion Matrix:
 [[1546   46   33]
 [  71  148   80]
 [  25   85  255]]
Validation MAE: 0.1739
Validation Accuracy: 0.8515
Test Confusion Matrix:
 [[755  30  14]
 [ 29  77  30]
 [ 15  48 147]]
Test MAE: 0.1703
Test Accuracy: 0.8550


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, mean_absolute_error, accuracy_score

train_data_df = pd.DataFrame(train_data)
test_data_df = pd.DataFrame(test_data)

train_target_df = pd.DataFrame(train_target)
test_target_df = pd.DataFrame(test_target)
# all_data = np.concatenate((train_data, test_data), axis=0)
all_data = pd.concat((train_data_df, test_data_df), axis=0)

all_target = pd.concat((train_target_df, test_target_df), axis=0)

# Convert targets to a numpy array for compatibility with sklearn
all_target = all_target.values.ravel()

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_data_df, train_target_df.values.ravel(), test_size=0.2, random_state=42)

# Define the MLP model
mlp = MLPClassifier(max_iter=1000, random_state=42)

# Define the parameter grid
param_grid = {
    'hidden_layer_sizes': [(50, 50), (100,), (100, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant', 'adaptive']
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit GridSearchCV to find the best parameters
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

# Evaluate the best model using the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)

# Calculate and print evaluation metrics
val_accuracy = accuracy_score(y_val, y_val_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_confusion_matrix = confusion_matrix(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation Confusion Matrix:\n{val_confusion_matrix}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=adam; total time=24.9min
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=adam; total time=29.8min
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=adam; total time=30.2min
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=adam; total time=31.3min
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=adam; total time=32.7min
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=adam; total time=35.4min
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=adam; total time=35.6min
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 50)

/mnt/storageG1/lwang/miniconda3/envs/ml-workshop/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(100,), learning_rate=constant, solver=sgd; total time=367.8min
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(50, 50), learning_rate=constant, solver=sgd; total time=285.2min
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100, 100), learning_rate=constant, solver=adam; total time=41.7min


/mnt/storageG1/lwang/miniconda3/envs/ml-workshop/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=adam; total time=61.5min
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=340.2min
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(50, 50), learning_rate=adaptive, solver=sgd; total time=370.7min


...

[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate=constant, solver=sgd; total time=217.4min
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100, 100), learning_rate=constant, solver=sgd; total time=163.3min
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100, 100), learning_rate=adaptive, solver=sgd; total time=152.0min


/mnt/storageG1/lwang/miniconda3/envs/ml-workshop/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate=adaptive, solver=sgd; total time=182.3min
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100, 100), learning_rate=adaptive, solver=sgd; total time=151.9min


/mnt/storageG1/lwang/miniconda3/envs/ml-workshop/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters found: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'sgd'}
Best cross-validation accuracy: 0.8402
Validation Accuracy: 0.8408
Validation MAE: 0.1929
Validation Confusion Matrix:
[[1249   41   34]
 [  69   78   69]
 [  28   52  220]]
